In [ ]:
import numpy as np

In [ ]:
help(np)

In [ ]:
a=[1,2,3]
print(np.exp(a))

In [ ]:
print(np.square(a))

In [ ]:
image = np.array([ 0.67826139,  0.29380381,344,333])

In [ ]:
print(image.reshape((image.shape[0],1)))
print(image.shape[0])

In [11]:
user=input("enter some  word")

enter some  wordi love you madan


In [12]:
ls=[]
ls=user.split()
print(ls)

['i', 'love', 'you', 'madan']


In [13]:
for i in ls:
    print(i)

i
love
you
madan


In [15]:
count=0
for i in ls:
    for j in i:
        if j=='i':
            count+=1
print(count)
        

1


In [18]:
number=input('enter any 10 number')


enter any 10 number1,2,3,4


In [20]:
ls=[]
greater=0
ls=number.split()
for i in ls:
     if int(i)>greater: 
            greater=i
print(greater)
    

ValueError: invalid literal for int() with base 10: '1,2,3,4'